## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-01-13-05-28 +0000,nyt,Zelensky Turns to Europe After Talks With U.S....,https://www.nytimes.com/2025/12/01/world/europ...
1,2025-12-01-13-03-27 +0000,bbc,Death toll in Indonesia floods passes 500 with...,https://www.bbc.com/news/articles/c4g4enlp6kzo...
2,2025-12-01-13-00-44 +0000,nyt,Markets Defy the White House,https://www.nytimes.com/2025/12/01/business/de...
3,2025-12-01-13-00-04 +0000,nyt,"OpenAI Takes Stake in Thrive Holdings, a Buyer...",https://www.nytimes.com/2025/12/01/business/de...
4,2025-12-01-12-59-45 +0000,nypost,Venezuela’s Maduro spotted for first time in d...,https://nypost.com/2025/12/01/us-news/venezuel...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2330/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
39,trump,22
379,new,10
103,party,9
120,dead,8
597,year,8


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
49,2025-12-01-11-00-00 +0000,latimes,"Attacks on ICE up 1,000%? Trump administration...",https://www.latimes.com/california/story/2025-...,59
152,2025-11-30-22-28-52 +0000,nypost,Why Venezuela’s allies Russia and China are sl...,https://nypost.com/2025/11/30/world-news/why-v...,55
114,2025-12-01-04-00-00 +0000,wsj,Economists predicted a global shock from Presi...,https://www.wsj.com/economy/trade/americas-tar...,53
79,2025-12-01-09-57-37 +0000,cbc,"Death toll rises, hundreds missing as Indonesi...",https://www.cbc.ca/news/world/death-toll-rises...,46
174,2025-11-30-20-36-42 +0000,cbc,Children among 4 dead in birthday party mass s...,https://www.cbc.ca/news/world/stockton-califor...,46


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
49,59,2025-12-01-11-00-00 +0000,latimes,"Attacks on ICE up 1,000%? Trump administration...",https://www.latimes.com/california/story/2025-...
79,46,2025-12-01-09-57-37 +0000,cbc,"Death toll rises, hundreds missing as Indonesi...",https://www.cbc.ca/news/world/death-toll-rises...
174,46,2025-11-30-20-36-42 +0000,cbc,Children among 4 dead in birthday party mass s...,https://www.cbc.ca/news/world/stockton-califor...
105,38,2025-12-01-05-28-53 +0000,nyt,Hong Kong’s Migrant Domestic Workers Mourn The...,https://www.nytimes.com/2025/11/30/world/asia/...
45,37,2025-12-01-11-00-00 +0000,nypost,Jersey City mayoral run-off between ex-NJ Gov....,https://nypost.com/2025/12/01/us-news/jersey-c...
144,35,2025-11-30-23-47-00 +0000,wsj,Congress launched inquiries and lawmakers from...,https://www.wsj.com/politics/national-security...
118,33,2025-12-01-02-55-07 +0000,nyt,Lebanon’s Christians Look to Pope Leo’s Visit ...,https://www.nytimes.com/2025/11/30/world/middl...
183,30,2025-11-30-19-34-17 +0000,nypost,White House names and shames ‘biased’ news out...,https://nypost.com/2025/11/30/us-news/white-ho...
4,28,2025-12-01-12-59-45 +0000,nypost,Venezuela’s Maduro spotted for first time in d...,https://nypost.com/2025/12/01/us-news/venezuel...
111,28,2025-12-01-04-17-59 +0000,nypost,6-year-old slapped in face during unprovoked a...,https://nypost.com/2025/11/30/us-news/boy-slap...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
